In [70]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [71]:
PSA = pd.read_csv('prices-split-adjusted.csv')

In [72]:
PSA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [73]:
PSA.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [74]:
PSA.symbol.size

851264

In [75]:
grpby = PSA.groupby('symbol')

In [133]:
WLTW = pd.DataFrame(grpby.get_group('WLTW'))

In [134]:
#having the date as the index makes it easier to access other fields
WLTW.set_index('date',inplace =True)

In [135]:
print(WLTW.loc['2016-01-06'])

symbol          WLTW
open          125.24
close         119.98
low           119.94
high          125.54
volume    2.3864e+06
Name: 2016-01-06, dtype: object


In [136]:
WLTW.loc['2016-01-06'].open

125.239998

In [137]:
whole_data = []
for date in WLTW.index:
    o = WLTW.loc[date].open
    c = WLTW.loc[date].close
    h = WLTW.loc[date].high
    l = WLTW.loc[date].low
    data = [o,c,h,l]
    whole_data.append(data)
#plt.boxplot(whole_data,labels=WLTW.index)
#plt.show()

In [138]:
whole_data1 = []
for date in WLTW.index[:30]:
    o = WLTW.loc[date].open
    c = WLTW.loc[date].close
    h = WLTW.loc[date].high
    l = WLTW.loc[date].low
    data = [o,c,h,l]
    whole_data1.append(data)
    
#plt.figure(figsize=(50,20))
#plt.boxplot(whole_data1,labels=WLTW.index[:30])
#x= WLTW.index[:30]
#plt.plot(x,8*np.log(WLTW.volume[x]),label='Volume')
#plt.plot(x,(WLTW.open[x] + (WLTW.close[x]-WLTW.open[x])/2),label='Avg')
#plt.legend()
#plt.show()

In [139]:
adl = []
obv = [0]
up_down = 1
x = WLTW.index[:30]
prev_date = x[0]
for date in x:
    #money flow multiplier
    mfm = ((WLTW.loc[date].close - WLTW.loc[date].low) - (WLTW.loc[date].high 
            - WLTW.loc[date].close))/(WLTW.loc[date].high - WLTW.loc[date].low)
    adl_n = mfm*WLTW.loc[date].volume
    adl.append(adl_n)
    if((WLTW.loc[date].close - WLTW.loc[prev_date].close) >= 0):
        up_down = 1
    else:
        up_down = -1
    obv_n = obv[-1] + up_down * WLTW.loc[date].volume
    obv.append(obv_n)
    prev_date = date
obv.remove(0)
#plt.figure(figsize=(50,20))
#plt.plot(x, adl,label='ADL')
#plt.bar(x,WLTW.volume[x], color = 'y')
#plt.plot(x,obv,color='r',label='OBV')
#plt.legend()
#plt.show()

In [183]:
def accDistLine(stockGroup):
    adl = []
    for date in stockGroup.index:
        o = stockGroup.loc[date].open
        c = stockGroup.loc[date].close
        h = stockGroup.loc[date].high
        l = stockGroup.loc[date].low
        #Money Flow Multiplier
        mfm = ((c-l)-(h-c))/(h-l)
        adl_n = mfm*stockGroup.loc[date].volume
        adl.append(adl_n)
    stockGroup['ADL'] = pd.Series(adl, index = stockGroup.index)

In [200]:
def obVolume(stockGroup):
    obv = [0]
    up_down = 1
    x = stockGroup.index
    prev_date = x[0]
    for date in x:
        if((stockGroup.loc[date].close - stockGroup.loc[prev_date].close) >= 0):
            up_down = 1
        else:
            up_down = -1
        obv_n = obv[-1] + up_down * stockGroup.loc[date].volume
        obv.append(obv_n)
        prev_date = date
    obv.remove(0)
    stockGroup['obV'] = pd.Series(obv, index = stockGroup.index)

In [197]:
def trueRangeCol(stockGroup):
    TR = []
    x = stockGroup.index
    for date in x:
        TR_n = stockGroup.loc[date].high - stockGroup.loc[date].low
        TR.append(TR_n)
    stockGroup['TR'] = pd.Series(TR, index = stockGroup.index)

In [186]:
def ATR(stockGroup):
    x = stockGroup.index
    ATR = []
    prev_date = x[13]
    atr14 = sum(stockGroup.TR[0:13])/14
    ATR.append(atr14)
    for date in x[14:]:
        ATR_n = (13 * stockGroup.TR[prev_date] + stockGroup.TR[date])/14
        ATR.append(ATR_n)
    stockGroup['ATR'] = pd.Series(ATR, index = stockGroup.index[13:])

In [213]:
def highLows(stockGroup):
    hph = []
    lpl = []
    x = stockGroup.index
    prev_date = x[0]
    for date in x[1:]:
        hph_n = stockGroup.high[date] - stockGroup.high[prev_date]
        lpl_n = stockGroup.low[date] - stockGroup.low[prev_date]
        hph.append(hph_n)
        lpl.append(lpl_n)
    stockGroup['hph'] = pd.Series(hph, index = stockGroup.index[1:])
    stockGroup['lpl'] = pd.Series(lpl, index = stockGroup.index[1:])

In [219]:
def smoothDX(stockGroup):
    x = stockGroup.index
    smoothDXplus = []
    for date in x:
        if ((stockGroup.hph[date] > stockGroup.lpl[date]) and stockGroup.hph[date] > 0):
            smoothDXplus.append(stockGroup.hph[date])
        else: 
            smoothDXplus.append(0)
    smoothDXminus = []
    for date in x:
        if ((stockGroup.lpl[date] > stockGroup.hph[date]) and stockGroup.lpl[date] > 0):
            smoothDXminus.append(stockGroup.lpl[date])
        else: 
            smoothDXminus.append(0)
    stockGroup['sDXm'] = pd.Series(smoothDXminus, index = stockGroup.index)
    stockGroup['sDXp'] = pd.Series(smoothDXplus, index = stockGroup.index)

In [207]:
def DMI(stockGroup):
    x = stockGroup.index
    DMIp = []
    DMIm = []
    for date in x[14:]:
        dmipn = stockGroup.sDXp[date]/stockGroup.ATR[date]
        dmimn = stockGroup.sDXm[date]/stockGroup.ATR[date]
        DMIp.append(dmipn)
        DMIm.append(dmimn)
    stockGroup['DMIp'] = pd.Series(DMIp, index = stockGroup.index[14:])
    stockGroup['DMIm'] = pd.Series(DMIm, index = stockGroup.index[14:])

In [247]:
def DX(stockGroup):
    DX = []
    x = stockGroup.index
    for date in x[14:]:
        dxn = 100.00 * abs(stockGroup.DMIp[date] - stockGroup.DMIm[date])/(stockGroup.DMIp[date] + stockGroup.DMIm[date])
        DX.append(dxn)
    stockGroup['DirX'] = pd.Series(DX, index = stockGroup.index[14:])

In [249]:
def ADX(stockGroup):
    #i have to put these into series and add to the main dataframe of the stock
    #also check the ranges on everything cause there'll be empty cells
    ADX = []
    prev_date = stockGroup.index[29]
    adx14 = sum(stockGroup.DirX[14:28])/14
    #ADX.append(adx14)
    for date in stockGroup.index[29:]:
        adxn = (stockGroup.DirX[prev_date] * 13 + stockGroup.DirX[date]) / 14
        ADX.append(adxn)
        prev_date = date
    stockGroup['ADX'] = pd.Series(ADX, index = stockGroup.index[29:])

In [192]:
#can separate and access things using %run notebookName.ipynb
#basically mimic a project

In [208]:
def setUp(sG):
    accDistLine(sG)
    trueRangeCol(sG)
    obVolume(sG)
    ATR(sG)
    highLows(sG)
    smoothDX(sG)
    DMI(sG)
    DX(sG)
    ADX(sG)

In [250]:
setUp(WLTW)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [254]:
WLTW = WLTW.fillna(0)

In [256]:
WLTW.head()

,symbol,open,close,low,high,volume,ADL,TR,obV,ATR,...,LpL,hph,lpl,sDXm,sDXp,DMIp,DMIm,DX,DirX,ADX
date,,,,,,,,,,,,,,,,,,,,,
2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0,1.713303e+06,3.940002,2163600.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0,-2.352308e+06,5.599999,-222800.0,0.0,...,-2.369996,-0.709999,-2.369996,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0,-2.468800e+06,4.809998,-2712300.0,0.0,...,-7.379998,-6.510002,-7.379998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0,1.171192e+06,3.940002,-706000.0,0.0,...,-8.809998,-8.809998,-8.809998,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0,-6.434316e+05,3.240006,-2114600.0,0.0,...,-8.220002,-8.919998,-8.220002,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [263]:
print(WLTW.close.rolling(5).mean())

date
2016-01-05           NaN
2016-01-06           NaN
2016-01-07           NaN
2016-01-08           NaN
2016-01-11    118.472000
2016-01-12    116.414001
2016-01-13    114.988000
2016-01-14    114.874000
2016-01-15    114.056000
2016-01-19    113.137999
2016-01-20    111.887999
2016-01-21    111.317999
2016-01-22    110.831999
2016-01-25    110.350000
2016-01-26    110.474001
2016-01-27    110.756000
2016-01-28    111.272000
2016-01-29    111.776001
2016-02-01    112.652000
2016-02-02    112.564000
2016-02-03    113.232001
2016-02-04    113.858000
2016-02-05    113.767999
2016-02-08    113.100000
2016-02-09    113.118001
2016-02-10    111.812000
2016-02-11    110.095999
2016-02-12    108.859999
2016-02-16    108.781998
2016-02-17    108.899997
                 ...    
2016-11-17    122.914000
2016-11-18    123.581999
2016-11-21    123.861999
2016-11-22    124.159999
2016-11-23    124.537999
2016-11-25    124.678000
2016-11-28    124.588001
2016-11-29    124.686000
2016-11-30    124.59